### Run for starting calculation
This is intended for starting the calculation from inside the docker container.
The resulting csv-files can be downloaded from city_results in the src directory.
This might take some time to work if you start the application for the first time as the db still needs to be imported.

In [ ]:
!python /safety_assessment/src/app/main.py

### Helper-functions for plotting results and analyzing results

In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from functools import reduce

# Load the infra-type results into a dataframe
df = pd.read_csv(f"../src/app/city_results/Stuttgart.csv") # <-----------Change the csv to match the city you want to analyse 

#Clean-up
df = df.drop(df[(df['avg_s_score'] < 0) | (df['avg_p_score'] < 0) | (df['avg_m_p_score'] < 0)].index)

# Comment this line, if the area you are analysing does not have enough data; we found count > 1000 to yield better results for Berlin
df = df.drop(df[df['count'] < 1000].index)
    
# Helper function for creating boxplots (one plot per infra-type) -> saves to .png in notebooks folder
def boxplot_helper():
    plt.rcParams["figure.figsize"] = (5,5)
    
    fig, axs = plt.subplots()

    data = [df['avg_p_score'], df['avg_s_score'], df['avg_m_p_score']]
    
    axs.boxplot(data)
    axs.set_yticks(np.arange(0, 1.01, step=0.2))
    axs.set_xticks(np.arange(0, 4, step=1))
    axs.set_xticklabels(['', 'p-score', 's-score', 'm-p-score'], rotation=45)
    
    plt.savefig(f'Boxplots.png', bbox_inches='tight', dpi=100)
    
# Helper function for creating bar-plots of every infra-type for a specified score with an average
def barplotWithAverage(score):
    mean = round(df[score].mean(), 4)
    
    plt.rcParams["figure.figsize"] = (10,15)
    fig, ax = plt.subplots()
    ax.barh(df.infra_type, df[score], align='center')
    ax.axvline(mean, color='black', linestyle='--')
    ax.set_yticks(df.infra_type)
    ax.invert_yaxis()
    ax.set_xticks(np.arange(0, 1.01, step=0.2))
    ax.set_xlabel(score)
    ax.annotate(f'Mean: {mean}', xy=(mean, 1), xytext=(-15, 15),
               xycoords=('data', 'axes fraction'), textcoords='offset points',
               horizontalalignment='right', verticalalignment='center', 
               arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0,
                            connectionstyle='angle,angleA=0,angleB=90,rad=10'),
               )
    
    plt.savefig(f'{score}-Barplots_With_Mean.png', bbox_inches='tight', dpi=100)

    
# Helper for only diplaying barplots for select infra-types with an average
def barplotSelectWithAverage(score, infra_types):
    dfp = reduce(pd.DataFrame.append, map(lambda i: df[df.infra_type == i], infra_types))
        
    mean = round(df[score].mean(), 4)
    
    plt.rcParams["figure.figsize"] = (5,7)
    fig, ax = plt.subplots()
    ax.barh(dfp.infra_type, dfp[score], align='center')
    ax.axvline(mean, color='black', linestyle='--')
    ax.set_yticks(dfp.infra_type)
    ax.invert_yaxis()
    ax.set_xticks(np.arange(0, 1.01, step=0.2))
    ax.set_xlabel(score)
    ax.annotate(f'Mean: {mean}', xy=(mean, 1), xytext=(-15, 15),
               xycoords=('data', 'axes fraction'), textcoords='offset points',
               horizontalalignment='right', verticalalignment='center', 
               arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0,
                            connectionstyle='angle,angleA=0,angleB=90,rad=10'),
               )
    
    plt.savefig(f'{score}-{infra_types[0]}-Barplots_With_Mean.png', bbox_inches='tight', dpi=100)

# Plots defined scores 
def curve2scores(score1, score2):
    fig,ax= plt.subplots()
    plt.rcParams["figure.figsize"] = (50,20)
    df3 = df.sort_values('infra_type')
    plt.plot(df3.infra_type, df3[score1], c='r', label=score1)
    plt.plot(df3.infra_type, df3[score2], c='b', label=score2)
    plt.title(f'{score1} and {score2} per infra-type')
    plt.legend(loc="lower left", prop={'size': 40})
    plt.xticks(rotation=90)
    ax.set_yticks(np.arange(0, 1.01, step=0.2))
    plt.savefig(f'{score1}_and_{score2}_Curves.png')

# Returns the n largest infra_types for specified score
def getNLargestInfraType(score, n):
    return df.nlargest(n, score, keep='first').infra_type

### Examples

In [ ]:
print(df['avg_p_score'].corr(df['avg_s_score']))

In [ ]:
print(getNLargestInfraType('avg_s_score', 3))

In [ ]:
boxplot_helper()

In [ ]:
curve2scores('avg_p_score', 'avg_s_score')

In [ ]:
barplotWithAverage('avg_p_score')

In [ ]:
barplotSelectWithAverage('avg_p_score', 
                         ['[bicycle = designated][segregated = yes]',
                          '[bicycle = designated][!segregated]',
                         ])